# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2019 Semester 1
-----
## Project 1: Gaining Information about Naive Bayes
-----
###### Student Name(s): Leonardo Linardi (855915)
###### Python version: 3
###### Submission deadline: 1pm, Fri 5 Apr 2019

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the five functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [1]:
# Importing external modules
import csv
from collections import defaultdict
import numpy as np
import math

In [2]:
def preprocess(filename):
    """
        // MAIN FUNCTION
        Opens a data file in csv specified by its 'filename' and transform it into a 'dataset' (list of list of instances).
    """
    lines = csv.reader(open(filename, "r"))
    dataset = list(lines)
    
    for i in range(len(dataset)):
        dataset[i] = [x for x in dataset[i]]
    
    return dataset

In [3]:
def get_class_labels(dataset):
    """
        // HELPER FUNCTION
        Given a 'dataset' (list of instances), returns the class label of all instances.     
    """
    dataset_array = np.array(dataset)
    class_labels = dataset_array[:,-1]
    return class_labels

In [4]:
def train_prior(dataset):
    """
        // HELPER FUNCTION
        Given a 'dataset' (list of instances), calculates the prior probabilities representing the 
        class distribution P(c). Returns a model (a dictionary, keys are class labels, values are counts or probabilities).
    """
    prior_prob = defaultdict(int)
    class_labels = get_class_labels(dataset)
    
    # calculate the frequency of each class label
    for lab in class_labels:
        prior_prob[lab] += 1
    
    # calculate the prior probability of each class label
    total_inst = len(dataset)
    for label in prior_prob:
        prior_prob[label] = prior_prob[label]/total_inst
    
    return prior_prob

In [5]:
def train_posterior(dataset):
    """
        // HELPER FUNCTION
        Given a 'dataset' (list of instances), calculates the posterior probabilities representing the 
        conditional probabilities P(a|c). Returns a model (a list (one per attribute) of lists (one per class) of
        dictionaries (keys are attribute values, values are counts or probabilities)).
    """
    posterior_count = []
    
    # stores the frequency of attributes per class label, for each attribute of the dataset
    for att in range(len(dataset[0])-1):
        class_dict = {}

        for inst in range(len(dataset)):
            att_value = dataset[inst][att]
            inst_label = dataset[inst][-1]
            
            # a dictionary hasn't exist yet for that label
            if inst_label not in class_dict:
                class_dict[inst_label] = defaultdict(int)
                class_dict[inst_label][att_value] += 1
            else:
                att_dict = class_dict[inst_label]
                att_dict[att_value] += 1
                class_dict[inst_label] = att_dict
        
        posterior_count.append(class_dict)
    
    return posterior_count

In [6]:
def train(dataset):
    """
        // MAIN FUNCTION
        Given a 'dataset' (list of instances), returns a 2-tuple of trained model, where
        a dictionary, keys are class labels, values are counts or probabilities
        prior_model     : a dictionary (keys are class labels, values are counts or probabilities) of probabilities 
                          (i.e. normalised counts) representing the class distribution P(c)
        posterior_model : a list (one per attribute) of lists (one per class) of dictionaries 
                          (keys are attribute values, values are counts or probabilities) representing the conditional 
                          probabilities P(a|c)
    """
    prior_model = train_prior(dataset)
    posterior_model = train_posterior(dataset)    
    return (prior_model, posterior_model)


In [7]:
def total_att_per_class(freqdict):
    """
        // HELPER FUNCTION
        Finds the total frequency of all attributes of a given class. 
    """
    total = 0
    for label, ct in freqdict.items():
        total += ct
    return total

In [8]:
def maximum_score(scores_dict):
    """
        // HELPER FUNCTION
        Finds the maximum score of class labels that a test instance could have,
        when predicting the label of that instance.
    """
    max_score = max(scores_dict.values())
    for label, ct in scores_dict.items():
        if ct == max_score:
            return label

In [9]:
EPSILON = 0.0001
CONSTANT_K = 0.1

def smoothing(label_counts_ddict, inst_att, method):
    """
        // HELPER FUNCTION
        Does the smoothing process upon calculating the frequency of attributes of a given class.
        The smoothing regime is done based on the method specified, which are:
        1: Epsilon Smoothing
        2: Laplace Smoothing (add-one smoothing)
        3: Add-k Smoothing
    """
    freq = label_counts_ddict[inst_att]
    total_freq = total_att_per_class(label_counts_ddict)
    num_of_att = len(label_counts_ddict)
    if method == 1:
        if freq == 0:
            freq = EPSILON
        return freq/total_freq
    elif method == 2:
        freq += 1
        total_freq += num_of_att
        return freq/total_freq
    elif method == 3:
        freq += CONSTANT_K
        total_freq += (num_of_att*CONSTANT_K)
        return freq/total_freq
    else:
        return 0

In [10]:
MISSING_VAL = '?'

def predict(model, testset):
    """
        // MAIN FUNCTION
        Predicts the class labels for each instance in a 'testset' (a list of instances), based on a 
        trained 'model' (2-tuple of prior and posterior model). Returns a list of labels that are predicted.
    """
    prior_model = model[0]
    posterior_model = model[1]
    predicted_labels = []
    
    # iterate through each test set, and determines which class label has the highest score
    for inst in testset:
        label_scores = {}
        for label in prior_model:
            curr_score = 0
            for att in range(len(inst)-1):
                att_value = inst[att]
                
                # ignores missing attribute values for test instances
                if att_value != MISSING_VAL:
                    label_counts_dict = posterior_model[att][label]
                    freq_count = smoothing(label_counts_dict, att_value, 3)
                    curr_score *= freq_count
                    
            # stores the score of the current label
            curr_score *= prior_model[label]
            label_scores[label] = curr_score
            
        predicted_labels.append(maximum_score(label_scores))
    
    return predicted_labels

In [11]:
def evaluate(model, testset):
    """
        // MAIN FUNCTION
        Evaluates a set of class label predictions, in a supervised context. Where the 'testset' is 
        a list of instances, based on a trained 'model' (2-tuple of prior and posterior model). 
        Uses the Accuracy method to evaluate the predictions.
    """
    actual_labels = get_class_labels(testset)
    predicted_labels = predict(model, testset)
    
    # evaluates the performance of the classifier
    correct = 0
    for i in range(len(actual_labels)):
        if actual_labels[i] == predicted_labels[i]:
            correct += 1
    accuracy = correct/len(actual_labels)
    print("Accuracy: ", accuracy)
    return

In [12]:
def info_gain(dataset):
    """
        // MAIN FUNCTION
        Given a 'dataset' (list of instances), this function calculates and returns a list of 
        Information Gain of each attribute.
    """
    prior_model = train(dataset)[0]
    all_info_gain = []
    
    for att in range(len(dataset[0])-1):
        
        # finds the class distribution of an attribute
        class_dist = {}
        for inst in range(len(dataset)):
            att_value = dataset[inst][att]
            inst_label = dataset[inst][-1]
            
            # a dictionary hasn't exist yet for that label
            if att_value not in class_dist:
                class_dist[att_value] = defaultdict(int)
                class_dist[att_value][inst_label] += 1 
            else:
                att_dict = class_dist[att_value]
                att_dict[inst_label] += 1
                class_dist[att_value] = att_dict
        
        info_gain = 0
        
        # calculate the entropy for all of the instances
        total_freq = total_att_per_class(prior_model)
        entropy = 0
        for cls, freq in prior_model.items():
            prob = freq/total_freq
            entropy += (prob)*(math.log(prob, 2))
        info_gain += (-entropy)
        
        # calculate the entropy of the class distribution
        mean_info = 0
        for att, ddict in class_dist.items():
            entropy = 0
            total_freq = total_att_per_class(ddict)
            
            for cls, freq in ddict.items():
                prob = freq/total_freq
                entropy += (prob)*(math.log(prob, 2))
            mean_info += (total_freq/len(dataset))*(-entropy)

        info_gain -= mean_info
        all_info_gain.append(info_gain)
        
    return all_info_gain

Questions (you may respond in a cell or cells below):

1. The Naive Bayes classifiers can be seen to vary, in terms of their effectiveness on the given datasets (e.g. in terms of Accuracy). Consider the Information Gain of each attribute, relative to the class distribution — does this help to explain the classifiers’ behaviour? Identify any results that are particularly surprising, and explain why they occur.
2. The Information Gain can be seen as a kind of correlation coefficient between a pair of attributes: when the gain is low, the attribute values are uncorrelated; when the gain is high, the attribute values are correlated. In supervised ML, we typically calculate the Infomation Gain between a single attribute and the class, but it can be calculated for any pair of attributes. Using the pair-wise IG as a proxy for attribute interdependence, in which cases are our NB assumptions violated? Describe any evidence (or indeed, lack of evidence) that this is has some effect on the effectiveness of the NB classifier.
3. Since we have gone to all of the effort of calculating Infomation Gain, we might as well use that as a criterion for building a “Decision Stump” (1-R classifier). How does the effectiveness of this classifier compare to Naive Bayes? Identify one or more cases where the effectiveness is notably different, and explain why.
4. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy. How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)
5. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the Naive Bayes classifier? Explain why, or why not.
6. Naive Bayes is said to elegantly handle missing attribute values. For the datasets with missing values, is there any evidence that the performance is different on the instances with missing values, compared to the instances where all of the values are present? Does it matter which, or how many values are missing? Would a imputation strategy have any effect on this?

Don't forget that groups of 1 student should respond to question (1), and one other question of your choosing. Groups of 2 students should respond to question (1) and question (2), and two other questions of your choosing. Your responses should be about 150-250 words each.